## Introduction to Text Mining and NLP: Term Paper

---------------------------
Ramón Talvi

David Vallmanya

Irene Villalonga

---------------------------

In this term paper we will use Reddit's API to extract a huge number of texts. We will focus on a community that provides creative writing prompts to inspire users to come up with mainly short stories. The prompt represents the main post and the comments to the prompt are the stories (in our case the documents). The main idea is to find the most prevalent topics in creative writing prompts on reddit. 

### 1) WEB SCRAPING

In [ ]:
import praw
import datetime
from praw.models import MoreComments

secret_token = "iIrB6d7nO_pdIDDqeTfc3p9l5F_xEw"
client_id = "QjdGILGBn4Tu9RXu0zyPvg"
username = "FabiBombo"
password = "Samba2323"

reddit = praw.Reddit(client_id=client_id,
                     client_secret=secret_token, password=password,
                     user_agent='MyBot/0.0.1', username=username)

#subreddit = reddit.subreddit('python')

In [3]:
limit_submission = 2000
limit_comments = 800

df = pd.DataFrame()

subreddit = reddit.subreddit('WritingPrompts')

hot_python = subreddit.top(limit=limit_submission, time_filter="all")

count_posts = 0
count_comments = 0
count_errors = 0

for submission in hot_python:
    if not submission.stickied:
        
        count_posts += 1
        
        p_awards = 0
        for award in submission.all_awardings:
             p_awards += 1
        
        for i in range(limit_comments+1):
            
            try:
                
                if submission.comments[i] == None:
                    continue
                
                if isinstance(submission.comments[i], MoreComments):
                    continue
                    
                if submission.comments[i].stickied == False:
                    
                                  
                    c_author = submission.comments[i].author
                    if c_author == None: 
                        c_author = "NA"
                    else:
                        c_author = c_author.name
                    
                    c_replies = 0
                    for reply in submission.comments[i].replies:
                        c_replies += 1
                        
                    c_awards = 0
                    for award in submission.comments[i].all_awardings:
                        c_awards += 1
                    
                    
                    df_comment = pd.DataFrame({
                        'post_id': submission.id,
                        'comment_id': submission.comments[i].id,
                        'subreddit': submission.subreddit.display_name,
                        'post_title': submission.title,
                        'post_ups': submission.ups,
                        'post_upvote_ratio': submission.upvote_ratio,
                        'post_num_comments': submission.num_comments,
                        'post_time': datetime.datetime.fromtimestamp(submission.created_utc),
                        'post_downs': submission.downs,
                        'post_score': submission.score,
                        'post_awards': p_awards,
                        'comment_awards': c_awards,
                        'comment_author': c_author,
                        'body': submission.comments[i].body,
                        'comment_replies': c_replies,
                        'comment_ups': submission.comments[i].ups,
                        'comment_downs': submission.comments[i].downs,
                        'comment_time' : datetime.datetime.fromtimestamp(submission.comments[i].created_utc),
                        'comment_score': submission.comments[i].score
                    }, index=[count_comments])
                    
                    df = pd.concat([df, df_comment])
                    
                    print("Post: "+str(count_posts) + "    Comment:"+ str(count_comments) + "    Inpost comment:" + str(i))
                    
                    count_comments += 1
                    
            except IndexError as e:
                count_errors += 1
                continue

print(count_errors)
df.to_csv("writingprompts_extract.csv", sep=';')
df

### 2) PRE PROCESSING 
In this section we are going to first load and clean the Data Frame. Then we will carry out the usual steps in Text Mining preprocessing: tokenization, lemmatizing, remove punctuation, unify, remove stopwords, stemming. Finally we will obtain the Document Term Matrix. 

In [4]:
import os
import pandas as pd

In [5]:
# Load dataframe

readin = '/Users/irenevillalonga/Desktop/Text Mining & NLP/Term paper/'
#spitout= '/Users/irenevillalonga/Desktop/Text Mining & NLP/Term paper/'
filename ="writingprompts_extract.csv"

df = pd.read_csv(os.path.join(readin, filename), sep=';', encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27960 entries, 0 to 27959
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         27960 non-null  object 
 1   post_id            27957 non-null  object 
 2   comment_id         27957 non-null  object 
 3   subreddit          27957 non-null  object 
 4   post_title         27957 non-null  object 
 5   post_ups           27957 non-null  float64
 6   post_upvote_ratio  27956 non-null  float64
 7   post_num_comments  27956 non-null  float64
 8   post_time          27956 non-null  object 
 9   post_downs         27956 non-null  float64
 10  post_score         27956 non-null  float64
 11  post_awards        27956 non-null  float64
 12  comment_awards     27956 non-null  float64
 13  comment_author     24882 non-null  object 
 14  body               27956 non-null  object 
 15  comment_replies    27955 non-null  float64
 16  comment_ups        279

In [6]:
# Remove NA

df = df.loc[df['body'] != '[removed]']
df = df.dropna(how='all')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27396 entries, 0 to 27959
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         27396 non-null  object 
 1   post_id            27393 non-null  object 
 2   comment_id         27393 non-null  object 
 3   subreddit          27393 non-null  object 
 4   post_title         27393 non-null  object 
 5   post_ups           27393 non-null  float64
 6   post_upvote_ratio  27392 non-null  float64
 7   post_num_comments  27392 non-null  float64
 8   post_time          27392 non-null  object 
 9   post_downs         27392 non-null  float64
 10  post_score         27392 non-null  float64
 11  post_awards        27392 non-null  float64
 12  comment_awards     27392 non-null  float64
 13  comment_author     24882 non-null  object 
 14  body               27392 non-null  object 
 15  comment_replies    27391 non-null  float64
 16  comment_ups        273

In [7]:
print("Title:", df.post_title[0])
print("   ")
print("Text:", df.body[0])

Title: [WP] The year is 1910. Adolf Hitler, a struggling artist, has fought off dozens of assasination attemps by well meaning time travelers, but this one is different. This traveller doesn't want to kill Hitler, he wants to teach him to paint. He pulls off his hood to reveal the frizzy afro of Bob Ross.
   
Text: There he sat, twirling his personal, stylized mustache. It was avant garde, just like he wanted to be. The man, as he was so, just wanted a place in this world for his art. He continues to stare at the easel, thinking. 

After a while he felt a firm, calming hand on his shoulder. He sighed, hanging his head wearily. "Are you yet another man come to end my life, if you can even see it that way?" The hand didn't answer, as it had no mouth. However, it's owner did, speaking the soft, assuaging tones that had come to make him famous. 

"No sir. I've seen too much death and war to want to do another such thing. Instead, I have come as a tutor. Here, grab that 2 inch brush and dip

In [8]:
# Esto serviria si cada texto tuviera un titulo.. Como tratamos esto?

In [9]:
import re
import nltk                                                             # Natural language Toolkit
from nltk.stem import SnowballStemmer                                   # Porter's II Stemmer
from nltk import word_tokenize                                          # Document tokenizer
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

porter=SnowballStemmer("english")
lmtzr = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))

from tqdm import tqdm
tqdm.pandas()

def abbr_or_lower(word):
    if re.match('([A-Z]+[a-z]*){2,}', word):
        return word
    else:
        return word.lower()

def tokenize(text, modulation):
    tokens = re.split(r'\W+', text)
    stems = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        lowers=abbr_or_lower(token)
        if lowers not in stop_words:
            if re.search('[a-zA-Z]', lowers):
                if modulation==1:
                    stems.append(porter.stem(lowers))
                if modulation==2:
                    stems.append(lmtzr.lemmatize(lowers))
                if modulation==0:
                    stems.append(lowers)
                stems.append(" ")
    return "".join(stems)

In [10]:
mod=1 #=1 means stemming, =2 means lemmatizing, =0 just lowercase
tit_preproc = (
    df.post_title
    .astype(str)
    .progress_apply(lambda row: tokenize(row, mod))
)

text_preproc = (
    df.body
    .astype(str)
    .progress_apply(lambda row: tokenize(row, mod))
)

df_proc = df
df_proc["titles_lemm"]=tit_preproc
df_proc["text_with_head_lemm"]=text_preproc

print("done with text plus titles")

df_proc = df_proc.dropna(subset=['titles_lemm']).reset_index(drop=True)

df_proc  = df_proc[['text_with_head_lemm', 'titles_lemm']]


100%|████████████████████████████████████| 27396/27396 [01:13<00:00, 372.75it/s]

done with text plus titles


In [11]:
for article in df.text_with_head_lemm[:1]:
    print(article)

sat twirl person styliz mustach avant gard like want man want place world art continu stare easel think felt firm calm hand shoulder sigh hang head wearili yet anoth man come end life even see way hand answer mouth howev owner speak soft assuag tone come make famous sir seen much death war want anoth thing instead come tutor grab inch brush dip titanium white prussian blue hitler thing man behind nod good mix togeth rather nice pale blue adolf brush stroke trembl across pallett easi tiger tri keep calm paint steadi confid adolf nod went time albeit bit slower mix anoth select done stranger pat shoulder good let see paint nice open sky bare paint ground let allow lie sigh man grab firm hold arm lift nice tini xs like portion top left hand corner soon fill nice layer blue go ahead tri adolf sigh attempt surpris mimick man stroke almost perfect ah ya go wash brush water beat devil wood point adolf help turn around surpris want beat easel afro figur behind nod pull woolen hood yes sir go h

In [12]:
Titles = df_proc.titles_lemm.tolist()
Texts = df_proc.text_with_head_lemm.tolist()

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

cv = CountVectorizer(ngram_range = (1,2))
cv.fit(Texts)
vectorized_text=cv.transform(Texts)
names=np.array(cv.get_feature_names())
print("dimensions of DT matrix are", vectorized_text.shape)

/Users/irenevillalonga/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


dimensions of DT matrix are (27396, 3041945)


In [14]:
vectorized_titles=cv.transform(Titles)

In [15]:
overall=vectorized_text+4*vectorized_titles 

In [16]:
overall

<27396x3041945 sparse matrix of type '<class 'numpy.int64'>'
	with 12018018 stored elements in Compressed Sparse Row format>

In [17]:
vectorized_text

<27396x3041945 sparse matrix of type '<class 'numpy.int64'>'
	with 11246895 stored elements in Compressed Sparse Row format>

### 3) POST PROCESSING
In this section we are going to use both the term frequency and the inverse document frequency to reweight the terms in the document term matrix. 

### 4) LDA IMPLEMENTATION 

### 5) LDA RESULT EVALUATION 